**LATAM GMV / MRR at OPP (Client/Location) Level**

5 Dec. 2022

Script to pull GMV and MRR data for all LATAM customers - per opportunity - in a DAILY basis for the last 2 months.

In [1]:
# Import packages, libraries and modules

#!pip install ds
import ds
import warnings
warnings.simplefilter("ignore")
import json
import os
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from ds.utilities.io import ds_trino
import numpy as np

# Display settings
from IPython.display import HTML, display
display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)

# Set max rows and columns to display
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

In [2]:
# Create connection to SuperSet via ds_trino
conn = ds_trino.create_trino_connection()

**Querying GMV data from superset**

- Data per opportunity per month
- Last 12 months

In [3]:
opty_query = """
   with ordenes as (



with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '2' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'LATAM' as region,
dc.country,



CASE 

when dc.country='Brazil' then 'Brazil'

else 'SSL'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
      facility_id in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when facility_id = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN facility_id = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
  -- WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when facility_id = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    WHEN facility_id = '812e6913-d78e-4dbc-877d-16993853aa7e' THEN 'SAO - Domingos'
  
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'

  
END
) AS facility_sf_name,
facility_name,
organization_id,
brand_id,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount,	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items

,customer_note

,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '2' month, '%Y-%m-%d')

AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type IN ('VIRTUAL', 'CLOUD_RETAIL')

--AND is_cancelled

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
           -- 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            --'6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            --'d65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
            --medellin
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f',--de moda
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            --'9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
           -- '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - av mexico
            
-- CHILE
        -- SANTIAGO
            --'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
             '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
             'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
            --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
            '14d9bb92-6376-4d69-adb6-64f97e4b3561',-- SAO - Ferreira
            '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
            
                
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                
                )


)


Select created_date,created_month, country, ordenes.organization_id, ordenes.facility_id, ordenes.facility_sf_name

,CASE 
when org.deleted_at is not null then True

ELSE  False END

as org_deleted_skusku





, count (DISTINCT (order_id)) as total_orders
, count(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders
,count (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders
,count (DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
,count(DISTINCT  CASE WHEN order_scanned_out THEN order_id END) as scanned_orders
,sum(subtotal) as total_gmv_local
,SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv_local
,count (DISTINCT ofo_slug) as OFOs
,count(DISTINCT city) as cities
, count(DISTINCT facility_sf_name) as facilities
, AVG (order_items_quantity) as Avg_Items_per_Order
, AVG(order_rating) as avg_rating
, AVG(in_handoff_at_ts-printed_at_ts) as avg_picking_time
,  avg(date_diff('minute',printed_at_ts,in_handoff_at_ts) )as avg_picking_time_minutes 
from ordenes

left join hudi_ingest.processed.store_config_organization_view as org

on ordenes.organization_id=org.organization_id

WHERE  row_number=1

and ordenes.organization_id not in (

'45ad476b-9874-4513-8a29-1b3b04180fbd', --casa de pastor
'2f3aefc9-2f28-45df-8de5-2b52530abef1', --bento vk
'cb671780-a757-4db5-8782-100106cad446', --delichurros vk
'842a7a9f-e25c-433d-bb4f-c1912db1bb04', --guisamor
'8d09b777-805e-49cf-a172-27fbf2840dd1',--donas americanas
'46f35c4f-1bf1-479d-92c9-e729a40ba816',-- FREE-TANGAS & GAR-NACHAS - VK
'66279914-31e6-4df9-9c57-36153545b28a',-- iRobot


--tests:

'beb89975-2feb-4d0b-a61e-ac2a3b87bd18', --Org Test PiknPak
'24622819-678c-45fb-a5be-acece3bf9c73', -- CR Brazil - Teste Superstore
'b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a', -- Conta Teste - Catherine
'fae923c9-8a1d-468e-9740-95c36c0fd4fc', --Conta Teste Pick n pack - Comercial
'804c6336-abc2-4293-81ee-0cea88c15029' -- warehouse and ops analyst
)



group by 1,2,3,4,5,6,7   


"""
print("Querying data...")
opp_data = ds_trino.fetch_data(conn=conn, sql_str= opty_query, use_cache=False)
print("Data pulled!")
print(len(opp_data))

Querying data...
Data pulled!
27427


In [4]:
# Inspect 
opp_data.head(3)

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None


In [5]:
opp_data.total_gmv_local.sum()

621137839.339941

Import customer names

In [6]:
client_query = """

SELECT organization_id,name 
FROM hudi_ingest.processed.store_config_organization_view

"""
print("Querying data...")
client_data = ds_trino.fetch_data(conn=conn, sql_str= client_query, use_cache=False)
print("Data pulled!")
print(len(client_data))

Querying data...
Data pulled!
339997


In [7]:
# Merge to opp data
opp_data = opp_data.merge(client_data, how = "left", on = "organization_id")

In [8]:
print(len(opp_data))

27427


In [9]:
opp_data.head()

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0,77,2716.44,0.00,1,1,1,2.181818,4.666667,None,None,D Doces - Pik n' Pak
4,2023-07-24,2023-07-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,59,2,0,2,59,17375.48,1860.00,1,1,1,1.813559,4.714286,None,None,Tere Cazola


Calculate metrics per customer per month

In [10]:
import pytz
import datetime
from datetime import timedelta

In [11]:
# Convert to time data the corrsponding columns

# Define a function to strip zeroes
def strip_zero(x):
    try:
        y=x.strip()
        return y[3:]
    except:
        return x

# Strip zeroes from avg_picking_time
opp_data["avg_picking_time"]= opp_data["avg_picking_time"].apply(strip_zero)

# Convert to datetime
opp_data['avg_picking_mins']=pd.to_datetime(opp_data['avg_picking_time'],errors='coerce').dt.minute+pd.to_datetime(opp_data['avg_picking_time'],errors='coerce').dt.second/60

In [12]:
# Create last updated column
opp_data["data_last_updated"] = datetime.datetime.now(pytz.timezone('Mexico/General'))

In [13]:
opp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27427 entries, 0 to 27426
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype                         
---  ------                    --------------  -----                         
 0   created_date              27427 non-null  object                        
 1   created_month             27427 non-null  object                        
 2   country                   27427 non-null  object                        
 3   organization_id           27427 non-null  object                        
 4   facility_id               27427 non-null  object                        
 5   facility_sf_name          27427 non-null  object                        
 6   org_deleted_skusku        27427 non-null  bool                          
 7   total_orders              27427 non-null  int64                         
 8   cancelled_orders          27427 non-null  int64                         
 9   issue_orders              27

In [14]:
# Convert string columns to numeric
opp_data["total_orders"] = pd.to_numeric(opp_data["total_orders"])
opp_data["cancelled_orders"] = pd.to_numeric(opp_data["cancelled_orders"])
opp_data["issue_orders"] = pd.to_numeric(opp_data["issue_orders"])
opp_data["ODR_orders"] = pd.to_numeric(opp_data["ODR_orders"])
opp_data["scanned_orders"] = pd.to_numeric(opp_data["scanned_orders"])

In [15]:
# Calculate ODR metrics
opp_data["pct_cancelled"] = opp_data["cancelled_orders"] / opp_data["total_orders"]
opp_data["pct_issues"] = opp_data["issue_orders"] / opp_data["total_orders"]
opp_data["pct_odr"] = opp_data["ODR_orders"] /  opp_data["total_orders"]
opp_data["pct_scanned"] = opp_data["scanned_orders"] / opp_data["total_orders"]

In [16]:
opp_data.head()

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0,77,2716.44,0.00,1,1,1,2.181818,4.666667,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000
4,2023-07-24,2023-07-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,59,2,0,2,59,17375.48,1860.00,1,1,1,1.813559,4.714286,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.033898,0.000000,0.033898,1.000000


In [17]:
# Inspect 
opp_data[opp_data["name"] == "Stuffed"].head(3)

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned


**Pull Salesforce Data for MRR**

In [18]:
# Install Simplesalesforce
# !pip install cryptography
from simple_salesforce import Salesforce, SalesforceLogin, SFType
import json

In [19]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [20]:
# Query category data from Salesforce
sf_query = """
SELECT 
Account.Org_Id_Otter__c,
Account_Name_Jira__c, 
account.Current_Customer__c, 
Opportunity_ID_18__c, 
Name, Facility__r.Name,  
Cuisine_Type__c,  
Id, 
Kitchen_City__c, 
Kitchen_Country__c, 
Is_Churned__c,
Churn_Date__c,
Processing_Fee__c, 
Monthly_Shelf_Fee_Dry_Storage__c,
Monthly_Shelf_Fee_Frozen_Storage__c,
Monthly_Shelf_Fee_Cold_Storage__c,
Closed_Won_Date__c,
Move_In_Date__c, 
Start_Date__c,
Loss_Reason__c, 
Specific_Lost_Reason__c, 
Lost_Explanation__c,
CR_Opp_Opted_out_Affiliate__c,
CR_Opp_Restrictions_Affiliate__c,
CR_Opp_Type_of_Processing_Fee__c,
account.CR_Account_Subcategory__c, 
account.CR_Acc_Social_Media_Presence_Numeric__c, 
account.CR_Type_of_Customer__c,
Storage_Amount_Dry_CU_FT__c, 
Storage_Amount_Frozen_CU_FT__c, 
Storage_Amount_Cold_CU_FT__c


FROM
Opportunity  

WHERE
RecordType.Name in ('Virtual', 'CloudRetail','virtual;cloudretail') AND (
Opp_Owner_Region__c in ('LATAM') OR Kitchen_Country__c IN ('Mexico', 'Colombia', 'Brazil'))
AND StageName = 'Closed Won' ORDER BY Closed_Won_Date__c DESC

"""


print("Querying data...")

# Query users to dataframe
sf_data = pd.DataFrame(sf.query_all(sf_query)['records'])
print("Data pulled!")

# Inspect 
sf_data.head()
print("Rows:", len(sf_data))

Querying data...
Data pulled!
Rows: 2775


In [21]:
# Inspect
sf_data.head(3)
print(len(sf_data))

2775


In [22]:
# Extract account details from accounts list in accounts column
account_name = [] # lista orgs id
for i in sf_data['Facility__r']:
    try:
        j=(i.get('Name'))
        #print(j)
        account_name.append(j)
    except:
        account_name.append(None)

# Append extracted lists to DF
sf_data['Facility__r']= account_name

In [23]:
# Rename new facility name field to match old name
sf_data.rename(columns = {"Facility__r":"Facility_Name__c"}, inplace = True)

In [24]:
sf_data.head(3)
print(len(sf_data))

2775


Query data from account table in SF

Removed CR_Acc_Social_Media_Presence__c to replace with a calculated bucket on march 2023

In [25]:
acct_query = """
SELECT
Account.LAM_TierScrape__c, 
Account.CSM__c,
Org_Id_Otter__c,
CR_Account_Categories__c,
CR_Account_Sub_category__c,
CR_Account_Cuisine_Type__c,
CR_Type_of_Customer__c,
CR_Acc_Social_Media_Presence_Numeric__c,
id

FROM
Account
"""


print("Querying data...")

# Query users to dataframe
acct_query = pd.DataFrame(sf.query_all(acct_query)['records'])
print("Data pulled!")

# Inspect 
acct_query.head()
print("Rows:", len(acct_query))

Querying data...
Data pulled!
Rows: 63801


In [26]:
acct_query.head(2)

,attributes,LAM_TierScrape__c,CSM__c,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,Id
0,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001VbfshQAB
1,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001Vbgz1QAB


In [27]:
# Create bucket for social media presence with same SF field name
social_media = []

for s in acct_query["CR_Acc_Social_Media_Presence_Numeric__c"]:
    if s >= 100000:
        s = '>100k'
    elif s >= 50000:
        s = '50k-100k'
    elif s >= 50000:
        s = '20k-50k'
    elif s >= 10000:
        s = '10k-20k'
    elif s >= 7000:
        s = '7k-10k'
    elif s >= 5000:
        s = '5k-7k'
    elif s >= 3000:
        s = '3k-5k'
    elif s >= 1000:
        s = '1k-3k'
    else:
        s = '<1k'
    social_media.append(s)

# Create new col with ex SF name
acct_query["CR_Acc_Social_Media_Presence__c"] = social_media

In [28]:
# Inspect
acct_query.head(3)

,attributes,LAM_TierScrape__c,CSM__c,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,Id,CR_Acc_Social_Media_Presence__c
0,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001VbfshQAB,<1k
1,"{'type': 'Account', 'url': '/services/data/v52...",None,None,None,None,None,None,None,NaN,0015G00001Vbgz1QAB,<1k
2,"{'type': 'Account', 'url': '/services/data/v52...",Tier 3,None,ab4eb478-b0bb-4fd5-8de9-8c3274b76911,CR - Frozen Food,CR - Meat/Shrimp/Chicken,CR - Baked Goods,None,114000.0,0015G00001VbnQEQAZ,>100k


In [29]:
acct_query = acct_query[["Org_Id_Otter__c", "CSM__c","CR_Account_Categories__c", "CR_Account_Sub_category__c", "CR_Account_Cuisine_Type__c", "CR_Acc_Social_Media_Presence__c", "LAM_TierScrape__c","CR_Type_of_Customer__c","CR_Acc_Social_Media_Presence_Numeric__c"]]
acct_query.head()

,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,None,None,None,None,None,<1k,None,None,NaN
1,None,None,None,None,None,<1k,None,None,NaN
2,ab4eb478-b0bb-4fd5-8de9-8c3274b76911,None,CR - Frozen Food,CR - Meat/Shrimp/Chicken,CR - Baked Goods,>100k,Tier 3,None,114000.0
3,None,None,None,None,None,<1k,None,None,NaN
4,None,None,None,None,None,<1k,None,None,NaN


In [30]:
acct_query = acct_query.drop_duplicates(subset = "Org_Id_Otter__c")

In [31]:
print(len(acct_query))

1781


In [32]:
# Extract account details from accounts list in accounts column
# Extract account details from accounts list in accounts column
org_ids = [] # lista orgs id
actual_cust =[] #lista cliente actual
has_churned = [] # lista has churned
for i in sf_data['Account']:
    try:
        j=(i.get('Org_Id_Otter__c'))
        #print(j)
        org_ids.append(j)
        k=(i.get('Current_Customer__c'))
        actual_cust.append(k)
    except:
        org_ids.append(None)
        actual_cust.append(None)

In [33]:
# Append extracted lists to DF
sf_data['sfdc_org_id']= org_ids
sf_data['current_customer']= actual_cust

In [34]:
# Fill NAs with bfill 
sf_data['org_id']= sf_data[["sfdc_org_id"]].bfill(axis=1).iloc[:, 0]

In [35]:
# Inspect
print("Rows:", len(sf_data))
sf_data.head()

Rows: 2775


,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",TEST DANIELA,0066T00001BfHs7QAF,TEST DANIELA - TP - Trial,LA - Hollywood (Western),None,0066T00001BfHs7QAF,Los Angeles,United States,False,None,NaN,NaN,NaN,NaN,None,None,2023-06-19,None,None,None,True,False,None,NaN,NaN,NaN,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,False,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Nieve De Olla,0066T00001C0a16QAB,Nieve De Olla - MEX - Roma,MEX - Roma,None,0066T00001C0a16QAB,Mexico City,Mexico,False,None,10.0,0.0,1250.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,a01f793e-ab3c-4de8-b99e-500be6d2977e,False,a01f793e-ab3c-4de8-b99e-500be6d2977e
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",La Licoreria Distribuidora,0066T00001D6dObQAJ,La Licoreria Distribuidora - MEX - Roma,MEX - Roma,None,0066T00001D6dObQAJ,Mexico City,Mexico,False,None,7.0,6000.0,0.0,0.0,None,None,None,None,None,None,True,False,Fixed Fee / Order,48.0,0.0,0.0,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,False,7d2329f1-fd48-49b5-8b1f-0400316f1ffd
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",The Healthy Bakery,0066T00001C0Zo2QAF,The Healthy Bakery - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0Zo2QAF,Mexico City,Mexico,False,None,9.0,0.0,2000.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,53f797b7-d31f-493f-a41f-c32ec09a0b92,False,53f797b7-d31f-493f-a41f-c32ec09a0b92
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",CUPCAKESLAND,0066T00001C0ZxiQAF,CUPCAKESLAND - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0ZxiQAF,Mexico City,Mexico,False,None,8.0,0.0,0.0,1000.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,0.0,4.0,c17f5cf9-88a9-445e-b777-58a88bf9efca,False,c17f5cf9-88a9-445e-b777-58a88bf9efca


In [36]:
# Merge data
sf_data = sf_data.merge(acct_query, how = "left", left_on = ["org_id"], right_on = ["Org_Id_Otter__c"])

In [37]:
print(len(sf_data))

2775


In [38]:
sf_data.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",TEST DANIELA,0066T00001BfHs7QAF,TEST DANIELA - TP - Trial,LA - Hollywood (Western),None,0066T00001BfHs7QAF,Los Angeles,United States,False,None,NaN,NaN,NaN,NaN,None,None,2023-06-19,None,None,None,True,False,None,NaN,NaN,NaN,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,False,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a,None,CR - Not Listed,None,CR - Not Listed,<1k,Tier 3,None,NaN
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Nieve De Olla,0066T00001C0a16QAB,Nieve De Olla - MEX - Roma,MEX - Roma,None,0066T00001C0a16QAB,Mexico City,Mexico,False,None,10.0,0.0,1250.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,a01f793e-ab3c-4de8-b99e-500be6d2977e,False,a01f793e-ab3c-4de8-b99e-500be6d2977e,a01f793e-ab3c-4de8-b99e-500be6d2977e,0055G0000080B5LQAU,CR - Frozen Desserts,CR - Shaved Ice,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",10k-20k,Tier 3,Local,24100.0
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",La Licoreria Distribuidora,0066T00001D6dObQAJ,La Licoreria Distribuidora - MEX - Roma,MEX - Roma,None,0066T00001D6dObQAJ,Mexico City,Mexico,False,None,7.0,6000.0,0.0,0.0,None,None,None,None,None,None,True,False,Fixed Fee / Order,48.0,0.0,0.0,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,False,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,7d2329f1-fd48-49b5-8b1f-0400316f1ffd,0055G000006QIsHQAW,CR - Alcoholic Beverages,CR - Spirits,None,<1k,Tier 2,Local,0.0
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",The Healthy Bakery,0066T00001C0Zo2QAF,The Healthy Bakery - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0Zo2QAF,Mexico City,Mexico,False,None,9.0,0.0,2000.0,0.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,4.0,0.0,53f797b7-d31f-493f-a41f-c32ec09a0b92,False,53f797b7-d31f-493f-a41f-c32ec09a0b92,53f797b7-d31f-493f-a41f-c32ec09a0b92,0055G0000080B5LQAU,CR - Baked Goods,CR - Cupcakes,CR - Baked Goods,10k-20k,Tier 2,Mid-Market,13800.0
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",CUPCAKESLAND,0066T00001C0ZxiQAF,CUPCAKESLAND - MEX - Insurgentes Sur,MEX - Insurgentes Sur,None,0066T00001C0ZxiQAF,Mexico City,Mexico,False,None,8.0,0.0,0.0,1000.0,None,None,2023-07-01,None,None,None,True,False,Percentage / Order,0.0,0.0,4.0,c17f5cf9-88a9-445e-b777-58a88bf9efca,False,c17f5cf9-88a9-445e-b777-58a88bf9efca,c17f5cf9-88a9-445e-b777-58a88bf9efca,0055G0000080B5LQAU,CR - Baked Goods,CR - Cupcakes,CR - Baked Goods,1k-3k,Tier 3,None,1663.0


**Calculating MRR**

- local currency
- merging fx rates and calculating mrr in usd

**Merging SF data to Superset Data**

In [39]:
# Order by closed_won date desc.
sf_data["Closed_Won_Date__c"] = pd.to_datetime(sf_data["Closed_Won_Date__c"])
sf_data = sf_data.sort_values(by = "Closed_Won_Date__c", ascending = False).reset_index(drop = True)

# Inspect 
sf_data.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001C0st4QAB,Shoppifit - MED - De Moda,MED - DModa,None,0066T00001C0st4QAB,Medellin,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kTaQAJ,Shoppifit - BOG - San Felipe,BOG - San Felipe,None,0066T00001D7kTaQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Food to Save,0066T00001D7txlQAB,Food to Save - BHZ - Paracatu,BHZ - Paracatu,None,0066T00001D7txlQAB,Sao Paulo,Brazil,False,None,0.0,0.0,0.0,0.0,2023-08-18,2023-07-26,2023-08-24,None,None,None,True,False,None,48.0,0.0,0.0,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,False,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,f9373a87-ab71-4198-bc2c-93fa2a27bcf8,0055G0000080AXnQAM,CR - Packaged Food / Ready to Eat,CR - Marmitas,CR - Packaged Food / Ready to Eat,>100k,Tier 1,Local,135000.0
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Shoppifit,0066T00001D7kShQAJ,Shoppifit - BOG - Atabanza,BOG - Atabanza,None,0066T00001D7kShQAJ,Bogota,Colombia,False,None,0.0,0.0,0.0,0.0,2023-08-18,None,2023-09-18,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,False,None,None,None,None,None,None,<1k,None,None,NaN
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Santa Rita ( helados y paletas ) - Empresa Activa,0066T00001D7ZWJQA3,Santa Rita ( helados y paletas ) - Empresa Act...,MEX - Polanco,None,0066T00001D7ZWJQA3,Mexico City,Mexico,False,None,8.0,0.0,1500.0,0.0,2023-08-17,None,2023-09-15,None,None,None,True,False,None,0.0,4.0,0.0,b368a673-374e-4990-b456-06fecf6ad9c9,False,b368a673-374e-4990-b456-06fecf6ad9c9,b368a673-374e-4990-b456-06fecf6ad9c9,None,CR - Frozen Desserts,CR - Ice Cream,None,10k-20k,None,Local,12000.0


In [40]:
# Dropping duplicates
print("Rows:", len(sf_data))
sf_data = sf_data.drop_duplicates(subset= ["org_id", "Facility_Name__c"], keep = "first").reset_index(drop=True)

Rows: 2775


In [41]:
# Merging SF to SS data
combined_data = opp_data.merge(sf_data,how='left', left_on=['organization_id',"facility_sf_name"], right_on=['org_id', "Facility_Name__c"])

In [42]:
# Inspect length
print("Rows:", len(opp_data))

Rows: 27427


In [43]:
# Creating MRR local column
combined_data["MRR_local"] = combined_data["Monthly_Shelf_Fee_Dry_Storage__c"] + combined_data["Monthly_Shelf_Fee_Frozen_Storage__c"] + combined_data["Monthly_Shelf_Fee_Cold_Storage__c"]

In [44]:
# Inspect length
print("Combined data rows:", len(combined_data))

Combined data rows: 27427


In [45]:
combined_data.head()

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,MEX - Roma,CR - Baked Goods,0065G00000j4xWsQAI,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,None,None,None,False,False,None,79.06,0.0,0.0,b556b267-8821-4162-a0cc-254fc70dafa0,True,b556b267-8821-4162-a0cc-254fc70dafa0,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",D'doces,0065G00000ouaslQAA,D'doces - SAO - Guararapes (Brooklin),SAO - Guararapes (Brooklin),CR - Baked Goods,0065G00000ouaslQAA,Sao Paulo,Brazil,False,None,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,None,None,None,False,False,None,4.00,0.0,4.0,e8683d22-ea10-4318-b400-1ff3b8b80338,True,e8683d22-ea10-4318-b400-1ff3b8b80338,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,930.0
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",D'doces,0065G00000dO1loQAC,D'doces - SAO - Dino - OE,SAO - Dino,CR - Baked Goods,0065G00000dO1loQAC,Sao Paulo,Brazil,False,None,5.00,0.0,0.0,2000.0,2021-08-30,2021-10-01,2021-10-01,None,None,None,False,False,None,0.00,0.0,20.0,e8683d22-ea10-4318-b400-1ff3b8b80338,True,e8683d22-ea10-4318-b400-1ff3b8b80338,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,2000.0
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0,77,2716.44,0.00,1,1,1,2.181818,4.666667,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000

**Calculating store health (GMV / MRR)**

**Importing and merging FX rates**

- MRR, GMV and health in USD

In [46]:
# Connect to google sheets exchange rate file
import pygsheets
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pandas_gbq
import requests

In [82]:
# Connecting and authenticating with credentials
s = secrets.Secrets()
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/cloud-platform')
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"), scopes = SCOPES)

gc = pygsheets.authorize(custom_credentials=credentials)

# Importing worksheet
ex = gc.open_by_key('1h_lgOO2wf89txM2jX5ZXS64DizqewyPUReI_0-zP2M0')
exws = ex.worksheet('title','latam_monthly')
latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')

In [48]:
# Inspecting
latam_monthly.head()

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.8
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.2
4,2019-05-01,Mexico,MXN,23.47


In [49]:
# Replacing NAs with 0 to enable conversion to numeric
latam_monthly['exchange_rate']= latam_monthly['exchange_rate'].replace('#N/A','0')
latam_monthly['exchange_rate']= pd.to_numeric(latam_monthly['exchange_rate'], errors='coerce')

In [50]:
# Merging FX rates to combined data
combined_data_fx = combined_data.merge(latam_monthly, how = "left", on = ['created_month','country'])

In [51]:
# Calculating GMV, MRR and Health in USD
combined_data_fx["mrr_usd"] = combined_data_fx["MRR_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["total_gmv_usd"] = combined_data_fx["total_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["cancelled_gmv_usd"] = combined_data_fx["cancelled_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["succesful_gmv_usd"] = combined_data_fx["total_gmv_usd"] - combined_data_fx["cancelled_gmv_usd"]
combined_data_fx["succesful_orders"] = combined_data_fx["total_orders"] - combined_data_fx["cancelled_orders"]

In [52]:
# Calculate total CU FT per opps
combined_data_fx["total_cu_ft"] = combined_data_fx["Storage_Amount_Dry_CU_FT__c"] + combined_data_fx["Storage_Amount_Frozen_CU_FT__c"] + combined_data_fx["Storage_Amount_Cold_CU_FT__c"]

In [53]:
# Examining data types
combined_data_fx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27427 entries, 0 to 27426
Data columns (total 77 columns):
 #   Column                                   Non-Null Count  Dtype                         
---  ------                                   --------------  -----                         
 0   created_date                             27427 non-null  object                        
 1   created_month                            27427 non-null  object                        
 2   country                                  27427 non-null  object                        
 3   organization_id                          27427 non-null  object                        
 4   facility_id                              27427 non-null  object                        
 5   facility_sf_name                         27427 non-null  object                        
 6   org_deleted_skusku                       27427 non-null  bool                          
 7   total_orders                             27427 no

**Calculating operations handling income**

- average processing fee % * succesful gmv

In [54]:
# Calculating income in local currency and USD
combined_data_fx["processing_fee_pct"] = combined_data_fx["Processing_Fee__c"] / 100
combined_data_fx["succesful_gmv_local"] = combined_data_fx["total_gmv_local"] - combined_data_fx["cancelled_gmv_local"]
combined_data_fx["processing_income_local"] = combined_data_fx["succesful_gmv_local"] * combined_data_fx["processing_fee_pct"]
combined_data_fx["processing_income_usd"] = combined_data_fx["succesful_gmv_usd"] * combined_data_fx["processing_fee_pct"]

In [55]:
# Create a concatenated field for name + facililty
combined_data_fx["name_facility"] = combined_data_fx["name"] + "-" + combined_data_fx["facility_sf_name"]

In [56]:
# Calculate org_health = gmv / mrr with local currency 
combined_data_fx["store_gmv_mrr"] = combined_data_fx["succesful_gmv_local"] / combined_data_fx["MRR_local"]

# Create org_health categories
health_cats = []
for i in combined_data_fx['store_gmv_mrr']:
    if i>=15:
        j='15x or more'
    elif i>=10:
        j='10x to 14.9x'
    elif i>=5:
        j='5x to 9.9x'
    elif i>=4:
        j='4x'
    elif i>=3:
        j='3x'
    elif i>=2:
        j='2x'
    elif i>=1:
        j='1x'
    else:
        j='0X'
    health_cats.append(j)

# Append categories
combined_data_fx["gmv_mrr_cat"] = health_cats

# Inspect
combined_data_fx.sample(3)

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
6788,2023-06-20,2023-06-01,Brazil,e59be2c8-b884-4e16-905f-e67f87c93a59,466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb,RIO - Barra,False,4,0,0,0,4,107.55,0.0,1,1,1,1.25,5.0,None,None,Cucurucho Doceria - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.0,0.0,0.0,1.0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Cucurucho Doceria,0066T00001BQHc2QAH,Cucurucho Doceria - RIO - Barra UPSELL,RIO - Barra,None,0066T00001BQHc2QAH,Rio de Janeiro,Brazil,True,2023-07-12,7.0,0.0,0.0,186.0,2023-03-31,2023-06-01,2023-06-01,None,None,None,False,False,Percentage / Order,0.0,0.0,4.0,e59be2c8-b884-4e16-905f-e67f87c93a59,True,e59be2c8-b884-4e16-905f-e67f87c93a59,e59be2c8-b884-4e16-905f-e67f87c93a59,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Baked Goods,10k-20k,Tier 2,Local,20100.0,186.0,BRL,4.86,38.271605,22.129630,0.0,22.129630,4,4.0,0.07,107.55,7.5285,1.549074,Cucurucho Doceria - Pik n' Pak-RIO - Barra,0.578226,0X
20462,2023-06-30,2023-06-01,Colombia,e218a8a1-c89a-44d8-b504-ce2622d562e1,58b818c6-615e-4316-b4a3-b9a6bb914c30,BAQ - Alto Prado,False,1,0,0,0,1,26000.00,0.0,1,1,1,1.00,NaN,None,None,Heladeria Magda,NaN,2023-08-22 17:07:05.343959-05:00,0.0,0.0,0.0,1.0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Heladeria Magda,0065G00000mS37oQAC,Heladeria Magda - BAQ - Alto Prado,BAQ - Alto Prado,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0065G00000mS37oQAC,Barranquilla,Colombia,False,None,8.0,0.0,400000.0,0.0,2022-01-05,2022-05-02,2022-02-05,None,None,None,False,False,None,0.0,8.0,0.0,e218a8a1-c89a-44d8-b504-ce2622d562e1,False,e218a8a1-c89a-44d8-b504-ce2622d562e1,e218a8a1-c89a-44d8-b504-ce2622d562e1,0055G0000080B5LQAU,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",7k-10k,Tier 3,None,9918.0,400000.0,COP,4204.31,95.140463,6.184130,0.0,6.184130,1,8.0,0.08,26000.00,2080.0000,0.494730,Heladeria Magda-BAQ - Alto Prado,0.065000,0X
21397,2023-07-30,2023-07-01,Brazil,21a0d55e-e0c3-4269-9e88-a22b178e3337,6c601837-403c-425f-a51c-31428ed1a67e,SAO - Rua do Acre,False,4,0,0,0,4,313.25,0.0,1,1,1,3.00,5.0,None,None,Comida Boa Low Carb,NaN,2023-08-22 17:07:05.343959-05:00,0.0,0.0,0.0,1.0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Comida Boa Low Carb Congelados,0065G00000mUWC3QAO,Comida Boa Low Carb Congelados - SAO - Rua do ...,SAO - Rua do Acre,CR - Packaged Food / Ready to Eat,0065G00000mUWC3QAO,Sao Paulo,Brazil,Fa

In [57]:
# calculating an estimated of daily MRR
combined_data_fx["mrr_daily_local"] = combined_data_fx["MRR_local"] / 30.416 #avg days per month
combined_data_fx["mrr_daily_usd"] = combined_data_fx["mrr_usd"] / 30.416

In [58]:
# Calculate org_health = gmv / mrr with local currency in daily basis
combined_data_fx["gmv_vs_mrr_daily"] = combined_data_fx["succesful_gmv_local"] / combined_data_fx["mrr_daily_local"]

# Create org_health categories
health_cats_daily = []
for i in combined_data_fx['gmv_vs_mrr_daily']:
    if i>=15:
        j='15x or more'
    elif i>=10:
        j='10x to 14.9x'
    elif i>=5:
        j='5x to 9.9x'
    elif i>=4:
        j='4x'
    elif i>=3:
        j='3x'
    elif i>=2:
        j='2x'
    elif i>=1:
        j='1x'
    else:
        j='0X'
    health_cats_daily.append(j)

# Append categories
combined_data_fx["gmv_mrr_cat_daily"] = health_cats_daily

# Inspect
combined_data_fx.sample(3)

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
15656,2023-06-22,2023-06-01,Mexico,5610229c-c4ce-4d72-94b3-4dc903b873d4,47e2d054-8b82-420c-b1b0-18010f57489c,MEX - Insurgentes Sur,False,1,0,0,0,1,165.22,0.0,1,1,1,3.0,NaN,None,None,Antonoff,NaN,2023-08-22 17:07:05.343959-05:00,0.0,0.0,0.0,1.0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Antonoff,0066T00001A5zcNQAR,Antonoff - MEX - Insurgentes Sur,MEX - Insurgentes Sur,CR - Baked Goods,0066T00001A5zcNQAR,Mexico City,Mexico,False,None,8.0,1500.0,0.0,2000.0,2022-12-30,2023-01-18,2023-02-01,None,None,None,False,False,Monthly Fixed Fee,4.0,0.0,12.0,5610229c-c4ce-4d72-94b3-4dc903b873d4,False,5610229c-c4ce-4d72-94b3-4dc903b873d4,5610229c-c4ce-4d72-94b3-4dc903b873d4,0055G000006QIsHQAW,CR - Baked Goods,CR - Cakes,CR - Baked Goods,5k-7k,Tier 1,Mid-Market,5682.0,3500.0,MXN,17.26,202.780997,9.572422,0.000000,9.572422,1,16.0,0.08,165.22,13.2176,0.765794,Antonoff-MEX - Insurgentes Sur,0.047206,0X,115.071015,6.666919,1.435809,1x
22568,2023-06-02,2023-06-01,Brazil,742598a4-f8a5-4d33-ab50-a1591c382c96,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,2,1,0,1,1,160.00,62.0,1,1,1,2.0,NaN,None,None,Coralina Bolos - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.5,0.0,0.5,0.5,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Coralina Bolos,0066T000018aJMsQAM,Coralina Bolos - RIO - Tijuca,RIO - Tijuca,CR - Baked Goods,0066T000018aJMsQAM,Rio de Janeiro,Brazil,False,None,7.0,464.0,928.0,NaN,2022-07-28,2023-02-03,2023-01-11,None,None,None,False,False,None,4.0,8.0,0.0,742598a4-f8a5-4d33-ab50-a1591c382c96,False,742598a4-f8a5-4d33-ab50-a1591c382c96,742598a4-f8a5-4d33-ab50-a1591c382c96,005f40000054nEXAAY,CR - Baked Goods,CR - Cakes,CR - Baked Goods,5k-7k,Tier 3,Local,6873.0,NaN,BRL,4.86,NaN,32.921811,12.757202,20.164609,1,12.0,0.07,98.00,6.8600,1.411523,Coralina Bolos - Pik n' Pak-RIO - Tijuca,NaN,0X,NaN,NaN,NaN,0X
5110,2023-08-16,2023-08-01,Mexico,c0b76b32-019e-49b5-9c27-b1cc2d042770,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,1,0,0,0,1,1008.00,0.0,1,1,1,1.0,NaN,None,None,Mezcal Amores,NaN,2023-08-22 17:07:05.343959-05:00,0.0,0.0,0.0,1.0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Mezcal Amores,0065G00000aCdkAQAS,Mezcal Amores - MEX - Roma,MEX - Roma,CR - Alcoholic Beverages,0065G00000aCdkAQAS,Mexico City,Mexico,False,None,5.0,3000.0,0.0,0.0,2021-06-07,2021-06-19,2

In [59]:
combined_data_fx[combined_data_fx['name']=='Tere Cazola'].head()

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.0,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,MEX - Roma,CR - Baked Goods,0065G00000j4xWsQAI,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,None,None,None,False,False,None,79.06,0.0,0.0,b556b267-8821-4162-a0cc-254fc70dafa0,True,b556b267-8821-4162-a0cc-254fc70dafa0,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,17.26,916.135574,2181.225377,139.629200,2041.596176,106,79.06,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more
4,2023-07-24,2023-07-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,59,2,0,2,59,17375.48,1860.0,1,1,1,1.813559,4.714286,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.033898,0.000000,0.033898,1.000000,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,MEX - Roma,CR - Baked Goods,0065G00000j4xWsQAI,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,None,None,None,False,False,None,79.06,0.0,0.0,b556b267-8821-4162-a0cc-254fc70dafa0,True,b556b267-8821-4162-a0cc-254fc70dafa0,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,16.93,933.992912,1026.313054,109.864146,916.448907,57,79.06,0.0351,15515.48,544.593348,32.167357,Tere Cazola-MEX - Roma,0.981216,0X,519.874408,30.707289,29.844670,15x or more
7,2023-07-27,2023-07-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,da27f06f-01fd-4106-88d9-b3fd41446515,MEX - Lindavista,False,37,1,0,1,35,9890.85,690.0,1,1,1,1.648649,NaN,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.027027,0.000000,0.027027,0.945946,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZrQAI,TERE CAZOLA LINDAVISTA - KITCHEN,MEX - Lindavista,CR - Bak

In [60]:
# Drop unused columns
combined_data_fx = combined_data_fx.drop(columns = ["attributes", "Account","sfdc_org_id", "org_id", "Facility_Name__c", "Id"])

# Inspect
combined_data_fx.head()

,created_date,created_month,country,organization_id,facility_id,facility_sf_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Cuisine_Type__c,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,CR - Baked Goods,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,None,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,17.26,916.135574,2181.225377,139.629200,2041.596176,106,79.06,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000,D'doces,0065G00000ouaslQAA,D'doces - SAO - Guararapes (Brooklin),CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,None,None,None,False,False,None,4.00,0.0,4.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,930.0,BRL,4.86,191.358025,196.757202,0.000000,196.757202,20,8.00,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538,D'doces,0065G00000dO1loQAC,D'doces - SAO - Dino - OE,CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,0.0,0.0,2000.0,2021-08-30,2021-10-01,2021-10-01,None,None,None,False,False,None,0.00,0.0,20.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,2000.0,BRL,4.86,411.522634,390.627572,12.135802,378.491770,51,20.00,0.0500,1839.47,91.973500,18.924588,D Doces - Pik n' Pak-SAO - Dino,0.919735,0X,65.754866,13.529808,27.974660,15x or more
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee6

In [61]:
# Renaming columns
combined_data_fx = combined_data_fx.rename(columns = {"facility_sf_name":"facility_name",
                                                      "organization_id": "org_id",
                                                      "Account_Name_Jira__c":"acct_name_jira",
                                                      "Kitchen_City__c":"kitchen_city",
                                                     "Kitchen_Country__c":"kitchen_country",
                                                     "Is_Churned__c":"is_churned",
                                                     "Churn_Date__c":"churn_date",
                                                     "Processing_Fee__c":"processing_fee_int",
                                                     "Closed_Won_Date__c":"closed_won_date"})

#Inspect
combined_data_fx.head()

,created_date,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Name,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,CR - Baked Goods,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,None,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,17.26,916.135574,2181.225377,139.629200,2041.596176,106,79.06,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000,D'doces,0065G00000ouaslQAA,D'doces - SAO - Guararapes (Brooklin),CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,None,None,None,False,False,None,4.00,0.0,4.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,930.0,BRL,4.86,191.358025,196.757202,0.000000,196.757202,20,8.00,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538,D'doces,0065G00000dO1loQAC,D'doces - SAO - Dino - OE,CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,0.0,0.0,2000.0,2021-08-30,2021-10-01,2021-10-01,None,None,None,False,False,None,0.00,0.0,20.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,2000.0,BRL,4.86,411.522634,390.627572,12.135802,378.491770,51,20.00,0.0500,1839.47,91.973500,18.924588,D Doces - Pik n' Pak-SAO - Dino,0.919735,0X,65.754866,13.529808,27.974660,15x or more
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0

In [62]:
# Filling NaNs with 0 on numeric columns
na_cols_gmv = ["MRR_local", "mrr_usd", "succesful_gmv_usd", "store_gmv_mrr", "gmv_vs_mrr_daily", "mrr_daily_local", "mrr_daily_usd"]

# Replacing NaNs
combined_data_fx[na_cols_gmv] = combined_data_fx[na_cols_gmv].fillna(0)

In [63]:
# Replacing inf values
combined_data_fx.replace([np.inf, -np.inf], 0, inplace=True)

In [64]:
combined_data_fx.head()

,created_date,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Name,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,Tere Cazola,0065G00000j4xWsQAI,TERE CAZOLA ROMA -KITCHEN,CR - Baked Goods,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,NaN,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,17.26,916.135574,2181.225377,139.629200,2041.596176,106,79.06,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000,D'doces,0065G00000ouaslQAA,D'doces - SAO - Guararapes (Brooklin),CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,NaN,None,None,False,False,None,4.00,0.0,4.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,930.0,BRL,4.86,191.358025,196.757202,0.000000,196.757202,20,8.00,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538,D'doces,0065G00000dO1loQAC,D'doces - SAO - Dino - OE,CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,0.0,0.0,2000.0,2021-08-30,2021-10-01,2021-10-01,NaN,None,None,False,False,None,0.00,0.0,20.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,2000.0,BRL,4.86,411.522634,390.627572,12.135802,378.491770,51,20.00,0.0500,1839.47,91.973500,18.924588,D Doces - Pik n' Pak-SAO - Dino,0.919735,0X,65.754866,13.529808,27.974660,15x or more
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0,77

In [65]:
# Drop unused columns
combined_data_fx = combined_data_fx.drop(columns = ["Name"])

# Inspect
combined_data_fx.head()

,created_date,created_month,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,pct_scanned,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,current_customer,Org_Id_Otter__c,CSM__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily
0,2023-06-18,2023-06-01,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.00,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.062500,0.955357,Tere Cazola,0065G00000j4xWsQAI,CR - Baked Goods,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,NaN,None,None,False,False,None,79.06,0.0,0.0,True,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,47800.0,15812.5,MXN,17.26,916.135574,2181.225377,139.629200,2041.596176,106,79.06,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more
1,2023-06-24,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.00,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.000000,1.000000,D'doces,0065G00000ouaslQAA,CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,NaN,None,None,False,False,None,4.00,0.0,4.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,930.0,BRL,4.86,191.358025,196.757202,0.000000,196.757202,20,8.00,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more
2,2023-06-29,2023-06-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,52,1,0,1,50,1898.45,58.98,1,1,1,2.230769,4.500000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.019231,0.000000,0.019231,0.961538,D'doces,0065G00000dO1loQAC,CR - Baked Goods,Sao Paulo,Brazil,False,None,5.00,0.0,0.0,2000.0,2021-08-30,2021-10-01,2021-10-01,NaN,None,None,False,False,None,0.00,0.0,20.0,True,e8683d22-ea10-4318-b400-1ff3b8b80338,0056T000009M15JQAS,CR - Baked Goods,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,20700.0,2000.0,BRL,4.86,411.522634,390.627572,12.135802,378.491770,51,20.00,0.0500,1839.47,91.973500,18.924588,D Doces - Pik n' Pak-SAO - Dino,0.919735,0X,65.754866,13.529808,27.974660,15x or more
3,2023-08-08,2023-08-01,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,87a40094-cf48-4032-8166-e4cee61bce2f,SAO - Dino,False,77,0,0,0,77,2716.44,0.00,1,1,1,2.181818,4.666667,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.34

In [66]:
combined_data_fx["total_gmv_usd"].sum()

2217975.430373601

In [67]:
# Converting corresponding columns to numeric and datetime
combined_data_fx["created_month"] = pd.to_datetime(combined_data_fx["created_month"])
combined_data_fx["total_orders"] = pd.to_numeric(combined_data_fx["total_orders"])
combined_data_fx["cancelled_orders"] = pd.to_numeric(combined_data_fx["cancelled_orders"])
combined_data_fx["issue_orders"] = pd.to_numeric(combined_data_fx["issue_orders"])
combined_data_fx["ODR_orders"] = pd.to_numeric(combined_data_fx["ODR_orders"])
combined_data_fx["scanned_orders"] = pd.to_numeric(combined_data_fx["scanned_orders"])
combined_data_fx["churn_date"] = pd.to_datetime(combined_data_fx["churn_date"])

**Uploading data to Google Sheets and Google Big Query**

**Renaming columns to new naming convention**

Nov. 2022

In [68]:
cols = list(combined_data_fx.columns.values)
cols

['created_date',
 'created_month',
 'country',
 'org_id',
 'facility_id',
 'facility_name',
 'org_deleted_skusku',
 'total_orders',
 'cancelled_orders',
 'issue_orders',
 'ODR_orders',
 'scanned_orders',
 'total_gmv_local',
 'cancelled_gmv_local',
 'OFOs',
 'cities',
 'facilities',
 'Avg_Items_per_Order',
 'avg_rating',
 'avg_picking_time',
 'avg_picking_time_minutes',
 'name',
 'avg_picking_mins',
 'data_last_updated',
 'pct_cancelled',
 'pct_issues',
 'pct_odr',
 'pct_scanned',
 'acct_name_jira',
 'Opportunity_ID_18__c',
 'Cuisine_Type__c',
 'kitchen_city',
 'kitchen_country',
 'is_churned',
 'churn_date',
 'processing_fee_int',
 'Monthly_Shelf_Fee_Dry_Storage__c',
 'Monthly_Shelf_Fee_Frozen_Storage__c',
 'Monthly_Shelf_Fee_Cold_Storage__c',
 'closed_won_date',
 'Move_In_Date__c',
 'Start_Date__c',
 'Loss_Reason__c',
 'Specific_Lost_Reason__c',
 'Lost_Explanation__c',
 'CR_Opp_Opted_out_Affiliate__c',
 'CR_Opp_Restrictions_Affiliate__c',
 'CR_Opp_Type_of_Processing_Fee__c',
 'Storage

In [69]:
# Selecting order to avoid affecting current MBR worksheet formulas
print(len(combined_data_fx))

# Order
combined_data_fx = combined_data_fx[['created_month',
                                     'created_date',
                                     'country',
                                     'org_id',
                                     'facility_id',
                                     'facility_name',
                                     'org_deleted_skusku',
                                     'total_orders',
                                     'cancelled_orders',
                                     'issue_orders',
                                     'ODR_orders',
                                     'scanned_orders',
                                     'total_gmv_local',
                                     'cancelled_gmv_local',
                                     'OFOs',
                                     'cities',
                                     'facilities',
                                     'Avg_Items_per_Order',
                                     'avg_rating',
                                     'avg_picking_time',
                                     'avg_picking_time_minutes',
                                     'name',
                                     'avg_picking_mins',
                                     'data_last_updated',
                                     'pct_cancelled',
                                     'pct_issues',
                                     'pct_odr',
                                     'acct_name_jira',
                                     'Opportunity_ID_18__c',
                                     'Cuisine_Type__c',
                                     'kitchen_city',
                                     'kitchen_country',
                                     'is_churned',
                                     'churn_date',
                                     'processing_fee_int',
                                     'Monthly_Shelf_Fee_Dry_Storage__c',
                                     'Monthly_Shelf_Fee_Frozen_Storage__c',
                                     'Monthly_Shelf_Fee_Cold_Storage__c',
                                     'closed_won_date',
                                     'Move_In_Date__c',
                                     'Start_Date__c',
                                     'Loss_Reason__c',
                                     'Specific_Lost_Reason__c',
                                     'Lost_Explanation__c',
                                     'CR_Opp_Opted_out_Affiliate__c',
                                     'CR_Opp_Restrictions_Affiliate__c',
                                     'CR_Opp_Type_of_Processing_Fee__c',
                                     'current_customer',
                                     #'Org_Id_Otter__c',  
                                     'MRR_local',
                                     'currency_code',
                                     'exchange_rate',
                                     'mrr_usd',
                                     'total_gmv_usd',
                                     'cancelled_gmv_usd',
                                     'succesful_gmv_usd',
                                     'processing_fee_pct',
                                     'succesful_gmv_local',
                                     'processing_income_local',
                                     'processing_income_usd',
                                     'name_facility',
                                     'store_gmv_mrr',
                                     'gmv_mrr_cat',
                                     'mrr_daily_local', 
                                     'mrr_daily_usd',
                                     'gmv_vs_mrr_daily', 
                                     'gmv_mrr_cat_daily',
                                     'succesful_orders',
                                     'total_cu_ft',
                                     'pct_scanned',
                                     'CSM__c',
                                     'CR_Account_Sub_category__c',
                                     'CR_Account_Cuisine_Type__c',
                                     'CR_Acc_Social_Media_Presence__c',
                                     'LAM_TierScrape__c',
                                     'CR_Type_of_Customer__c',
                                     'CR_Account_Categories__c',
                                     'CR_Acc_Social_Media_Presence_Numeric__c']]

27427


In [70]:
# Inspect
combined_data_fx.head(2)

,created_month,created_date,country,org_id,facility_id,facility_name,org_deleted_skusku,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating,avg_picking_time,avg_picking_time_minutes,name,avg_picking_mins,data_last_updated,pct_cancelled,pct_issues,pct_odr,acct_name_jira,Opportunity_ID_18__c,Cuisine_Type__c,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,current_customer,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,mrr_daily_local,mrr_daily_usd,gmv_vs_mrr_daily,gmv_mrr_cat_daily,succesful_orders,total_cu_ft,pct_scanned,CSM__c,CR_Account_Sub_category__c,CR_Account_Cuisine_Type__c,CR_Acc_Social_Media_Presence__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Account_Categories__c,CR_Acc_Social_Media_Presence_Numeric__c
0,2023-06-01,2023-06-18,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.0,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.0625,Tere Cazola,0065G00000j4xWsQAI,CR - Baked Goods,Mexico City,Mexico,False,NaT,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,NaN,None,None,False,False,None,True,15812.5,MXN,17.26,916.135574,2181.225377,139.6292,2041.596176,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more,106,79.06,0.955357,005f4000006BISzAAO,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,CR - Baked Goods,47800.0
1,2023-06-01,2023-06-24,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.0,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.0000,D'doces,0065G00000ouaslQAA,CR - Baked Goods,Sao Paulo,Brazil,False,NaT,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,NaN,None,None,False,False,None,True,930.0,BRL,4.86,191.358025,196.757202,0.0000,196.757202,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more,20,8.00,1.000000,0056T000009M15JQAS,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,CR - Baked Goods,20700.0


In [71]:
# Renaming to new convention
combined_data_fx_2 = combined_data_fx.rename(columns={'created_month' : "A_Created_Month",
                                                     'created_date' : "A_Created_Date",
                                                     'country' : "Country",
                                                     'org_id' : "Organization_Id",
                                                     'facility_id' : "Facility_Id",
                                                     'facility_name' : "Facility_Name",
                                                     'org_deleted_skusku' : "Org_Deleted_SKU",
                                                     'total_orders' : "Orders_All",
                                                     'cancelled_orders' : "Orders_Cancelled",
                                                     'issue_orders' : "Orders_Issue",
                                                     'ODR_orders' : "Orders_ODR",
                                                     'scanned_orders' : "Orders_Scanned",
                                                     'total_gmv_local' : "GMV_Loc_All",
                                                     'cancelled_gmv_local' : "GMV_Loc_Cancelled",
                                                     'OFOs' : "Active_OFOs",
                                                     'cities' : "Active_Cities",
                                                     'facilities' : "Active_Facilities",
                                                     'Avg_Items_per_Order' : "Avg_SKUs_Order",
                                                     'avg_rating' : "Avg_Rating",
                                                     'avg_picking_time' : "Avg_PickingTimestamp",
                                                     'avg_picking_time_minutes' : "Avg_PickingMinutes",
                                                     'name' : "Account_Name",
                                                     'avg_picking_mins' : "Avg_Picking_Mins_Calc",
                                                     'data_last_updated' : "Data_Last_Updated",
                                                     'pct_cancelled' : "Orders_Perc_Cancelled",
                                                     'pct_issues' : "Orders_Perc_Issues",
                                                     'pct_odr' : "Orders_Perc_ODR",
                                                     'acct_name_jira' : "SF_Account_Name",
                                                     'Opportunity_ID_18__c' : "SF_Opportunity_Id",
                                                     'Cuisine_Type__c' : "SF_Cuisine_Type",
                                                     'kitchen_city' : "SF_City",
                                                     'kitchen_country' : "SF_Country",
                                                     'is_churned' : "SF_Is_Churned",
                                                     'churn_date' : "SF_Churn_Date",
                                                     'processing_fee_int' : "SF_Processing_Fee_int",
                                                     'Monthly_Shelf_Fee_Dry_Storage__c' : "SF_Dry_Storage_Fee",
                                                     'Monthly_Shelf_Fee_Frozen_Storage__c' : "SF_Frozen_Storage_Fee",
                                                     'Monthly_Shelf_Fee_Cold_Storage__c' : "SF_Cold_Storage_Fee",
                                                     'closed_won_date' : "SF_Closed_Won_Date",
                                                     'Move_In_Date__c': "SF_Move_In_Date",
                                                     'Start_Date__c' : "SF_Start_Date",
                                                     'Loss_Reason__c' : "SF_Loss_Reason",
                                                     'Specific_Lost_Reason__c' : "SF_Specific_Loss_Reason",
                                                     'Lost_Explanation__c' : "SF_Loss_Explanation",
                                                     'CR_Opp_Opted_out_Affiliate__c': "SF_Opp_Opted_Out_Affiliate",
                                                     'CR_Opp_Restrictions_Affiliate__c' : "SF_Opp_Restrictions_Affiliate",
                                                     'CR_Opp_Type_of_Processing_Fee__c' : "SF_Opp_Type_of_Processing_Fee",
                                                     'current_customer' : "SF_Current_Customer",
                                                     #'Org_Id_Otter__c',  
                                                     'MRR_local' : "SF_MRR_Loc",
                                                     'currency_code' : "Finance_Currency",
                                                     'exchange_rate' : "Finance_FX",
                                                     'mrr_usd' : "SF_MRR_USD",
                                                     'total_gmv_usd' : "GMV_USD_All",
                                                     'cancelled_gmv_usd' : "GMV_USD_Cancelled",
                                                     'succesful_gmv_usd' : "GMV_USD_Successful",
                                                     'processing_fee_pct' : "SF_Processing_Fee_Perc",
                                                     'succesful_gmv_local' : "GMV_Loc_Successful",
                                                     'processing_income_local' : "Income_PF_Loc",
                                                     'processing_income_usd' : "Income_PF_USD",
                                                     'name_facility' : "Concat_AccName_Facility",
                                                     'store_gmv_mrr' : "GMV_vs_MRR",
                                                     'gmv_mrr_cat' : "GMV_MRR_Group",
                                                     'mrr_daily_local' : "SF_MRR_DAILY_LOC", 
                                                     'mrr_daily_usd' : "SF_MRR_DAILY_USD",
                                                     'gmv_vs_mrr_daily' : "GMV_vs_MRR_DAILY", 
                                                     'gmv_mrr_cat_daily' : "GMV_MRR_DAILY_GROUP",
                                                     'succesful_orders' : "Orders_Successful",
                                                     'total_cu_ft' : "SF_Storage_Cuft",
                                                     'pct_scanned' : "Orders_Perc_Scanned",
                                                     'CSM__c' : "SF_CSM",
                                                     'CR_Account_Sub_category__c' : "SF_Subcategory",
                                                     'CR_Account_Cuisine_Type__c' : "SF_Account_Cuisine_Type",
                                                     'CR_Acc_Social_Media_Presence__c' : "SF_SocialMediaPresence",
                                                     'LAM_TierScrape__c' : "SF_TierScrape",
                                                     'CR_Type_of_Customer__c' : "SF_TypeofCustomer",
                                                     'CR_Account_Categories__c' : "SF_Category",
                                                     'CR_Acc_Social_Media_Presence_Numeric__c':'SF_SocialMediaPresence_Numeric'})

In [72]:
# Inspect
combined_data_fx_2.head(2)

,A_Created_Month,A_Created_Date,Country,Organization_Id,Facility_Id,Facility_Name,Org_Deleted_SKU,Orders_All,Orders_Cancelled,Orders_Issue,Orders_ODR,Orders_Scanned,GMV_Loc_All,GMV_Loc_Cancelled,Active_OFOs,Active_Cities,Active_Facilities,Avg_SKUs_Order,Avg_Rating,Avg_PickingTimestamp,Avg_PickingMinutes,Account_Name,Avg_Picking_Mins_Calc,Data_Last_Updated,Orders_Perc_Cancelled,Orders_Perc_Issues,Orders_Perc_ODR,SF_Account_Name,SF_Opportunity_Id,SF_Cuisine_Type,SF_City,SF_Country,SF_Is_Churned,SF_Churn_Date,SF_Processing_Fee_int,SF_Dry_Storage_Fee,SF_Frozen_Storage_Fee,SF_Cold_Storage_Fee,SF_Closed_Won_Date,SF_Move_In_Date,SF_Start_Date,SF_Loss_Reason,SF_Specific_Loss_Reason,SF_Loss_Explanation,SF_Opp_Opted_Out_Affiliate,SF_Opp_Restrictions_Affiliate,SF_Opp_Type_of_Processing_Fee,SF_Current_Customer,SF_MRR_Loc,Finance_Currency,Finance_FX,SF_MRR_USD,GMV_USD_All,GMV_USD_Cancelled,GMV_USD_Successful,SF_Processing_Fee_Perc,GMV_Loc_Successful,Income_PF_Loc,Income_PF_USD,Concat_AccName_Facility,GMV_vs_MRR,GMV_MRR_Group,SF_MRR_DAILY_LOC,SF_MRR_DAILY_USD,GMV_vs_MRR_DAILY,GMV_MRR_DAILY_GROUP,Orders_Successful,SF_Storage_Cuft,Orders_Perc_Scanned,SF_CSM,SF_Subcategory,SF_Account_Cuisine_Type,SF_SocialMediaPresence,SF_TierScrape,SF_TypeofCustomer,SF_Category,SF_SocialMediaPresence_Numeric
0,2023-06-01,2023-06-18,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,bcb04949-3cda-4ca0-be39-b9f579e356ca,MEX - Roma,False,112,6,1,7,107,37647.95,2410.0,1,1,1,1.794643,4.863636,None,None,Tere Cazola,NaN,2023-08-22 17:07:05.343959-05:00,0.053571,0.008929,0.0625,Tere Cazola,0065G00000j4xWsQAI,CR - Baked Goods,Mexico City,Mexico,False,NaT,3.51,15812.5,0.0,0.0,2021-11-19,2021-11-01,2021-11-01,NaN,None,None,False,False,None,True,15812.5,MXN,17.26,916.135574,2181.225377,139.6292,2041.596176,0.0351,35237.95,1236.852045,71.660026,Tere Cazola-MEX - Roma,2.228487,2x,519.874408,30.120186,67.781659,15x or more,106,79.06,0.955357,005f4000006BISzAAO,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,CR - Baked Goods,47800.0
1,2023-06-01,2023-06-24,Brazil,e8683d22-ea10-4318-b400-1ff3b8b80338,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,20,0,0,0,20,956.24,0.0,1,1,1,2.850000,5.000000,None,None,D Doces - Pik n' Pak,NaN,2023-08-22 17:07:05.343959-05:00,0.000000,0.000000,0.0000,D'doces,0065G00000ouaslQAA,CR - Baked Goods,Sao Paulo,Brazil,False,NaT,5.00,465.0,0.0,465.0,2022-06-07,2022-07-26,2022-08-15,NaN,None,None,False,False,None,True,930.0,BRL,4.86,191.358025,196.757202,0.0000,196.757202,0.0500,956.24,47.812000,9.837860,D Doces - Pik n' Pak-SAO - Guararapes (Brooklin),1.028215,1x,30.576013,6.291361,31.274189,15x or more,20,8.00,1.000000,0056T000009M15JQAS,CR - Cakes,CR - Baked Goods,10k-20k,Tier 1,None,CR - Baked Goods,20700.0


In [73]:
# Checking data types
combined_data_fx_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27427 entries, 0 to 27426
Data columns (total 77 columns):
 #   Column                          Non-Null Count  Dtype                         
---  ------                          --------------  -----                         
 0   A_Created_Month                 27427 non-null  datetime64[ns]                
 1   A_Created_Date                  27427 non-null  object                        
 2   Country                         27427 non-null  object                        
 3   Organization_Id                 27427 non-null  object                        
 4   Facility_Id                     27427 non-null  object                        
 5   Facility_Name                   27427 non-null  object                        
 6   Org_Deleted_SKU                 27427 non-null  bool                          
 7   Orders_All                      27427 non-null  int64                         
 8   Orders_Cancelled                27427 non-null

**Upload to tests spread**

# Uploading to Google Sheets # Test
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(service_file= "/home/alan.tang/Creds/creds_spread_at.json")
print("Opening Spreadsheet..")
ss = gc.open_by_key("1OFg4SVVddVWPJaWARUVN4Ifimqpn0vx6neytKWOTegg")

# Selecting location
ws = ss.worksheet("title","opps_daily")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="CA")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

**MRR / GMV report uploading to WBR File**

In [75]:
# Uploading to Google Sheets
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("1YxhDbhtDPobaIjDToqhy2i7GYXZRHczS5IYIag40KZs")

# Selecting location
ws = ss.worksheet("title","D_GMV_MRR_Opp")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end= "BY")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(combined_data_fx_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


Upload to GBQ

In [80]:
# Import libraries
from pandas.io import gbq
from google.oauth2 import service_account
from google.cloud import bigquery

In [ ]:
# Connecting and authenticating with credentials
s = secrets.Secrets()
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/cloud-platform')
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"), scopes = SCOPES)

In [83]:
# Upload data to GBQ (original DB without the new names and columns nov 22)

print("Uploading data to GBQ...")

table_id = 'css-operations.latam_cr_bizops.gmv_mrr_opp_daily'
project_id = 'css-operations'
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"), scopes = SCOPES)
pandas_gbq.to_gbq(combined_data_fx_2,table_id, project_id = project_id, if_exists = 'replace', credentials = credentials)

print("Data uploaded!")

Uploading data to GBQ...


1it [00:06,  6.77s/it]

Data uploaded!
